In [1]:
from promisces import *
import numpy as np
from copy import deepcopy

#  Example: PFAS in the "semi-closed water cycle"

The semi-closed water cycle is a combination of wastewater treatment, dilutions processes in surface water and groundwater, bank filtration and drinking water treatment.

In this example the exposure to 14 substancesis simulated. The data sent contains removal data for one of them (PFOA) as an example. Additional literature data can manually be added. However, the simulation even runs without literature data, using the prior information only. Case study specific data, like on-site measurements, can be added to further refine the results.

To run a simulation, a starting concentration and a tolerable final concentration (reference value) must be provided. In case of mixture processes, like dilution by surface water, this information is needed additionally.  

## 1. Enter starting concentration and reference value per substance
### Starting concentrations
As a starting concentration the maximum value from the PROMISCES HHEA Dataset was used per substance in raw wastewter.

### Reference values
Reference values for the single PFAS substances are derived from the threshold of PFAS-20 sum according to the European drinking water directive. The threshold value for all 20 PFAS is 100 ng/L, equally distributed among the 20 substances that leads to 5 ng/L per substance. However, substances have a different toxicity which need to be considered. This can be done by using Relative Potancy Factors (RPFs) in relation to PFOA toxicity (ex. the RPF of PFOA and PFOS is 1 and 2, respectively. That leads to single compound threshold of 5 ng/L and 2.5 ng/L)
The RPFs were taken from:

"RIVM 2025":https://www.rivm.nl/pfas/rpf
(accessed 2025-04-15)

based on 

[1] Bil W, Zeilmaker M, Fragki S, Lijzen J, Verbruggen E, Bokkers B. 2021. Risk assessment of per‐ and polyfluoroalkyl substance mixtures: a relative potency factor approach. Environ Toxicol Chem 40 (3): 859–870. 

[2] RIVM. 2022. RIVM-VSP (Veiligheid Stoffen en Producten) Advies 14434A02–Drinkwaterrichtwaarde voor trifluorazijnzuur, 02-02-2022. 

[3] Smit CE (Conformité Européenne), Verbruggen EMJ. 2022. Risicogrenzen voor PFAS in oppervlaktewater. Doorvertaling van de gezondheidskundige grenswaarde van EFSA (Europese Voedselveiligheidsautoriteit) naar concentraties in water. RIVM Briefrapport 2022-0074.  


### Special case PFPeS and TFMSA
To possible solutions to include substance without any knowledge about their concentration:
 - For PFPeS there was no concentration in raw wastewater available. Instead a concentration 100 times the reference value was used. 

 - For TFMSA neither starting concentration nor reference value were available. It was decided to conduct a relative assessment where reference value and starting conentration were set to 1 and 100, respectively.


In [2]:
# substances
substances = [
    Substances.pfba \
        .with_starting_concentration(StartingConcentration(np.array([19]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 100, 2023, "")),
    Substances.pfpea \
        .with_starting_concentration(StartingConcentration(np.array([27]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 100, 2023, "")),
    Substances.pfhxa \
        .with_starting_concentration(StartingConcentration(np.array([270]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 500, 2023, "")),
    Substances.pfhpa \
        .with_starting_concentration(StartingConcentration(np.array([58]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 5, 2023, "")),
    Substances.pfoa \
        .with_starting_concentration(StartingConcentration(np.array([550]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 5, 2023, "")),
    Substances.pfna \
        .with_starting_concentration(StartingConcentration(np.array([26.4]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 0.5, 2023, "")),
    Substances.pfda \
        .with_starting_concentration(StartingConcentration(np.array([27]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 0.5, 2023, "")),
    Substances.pfbs \
        .with_starting_concentration(StartingConcentration(np.array([37]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 5000, 2023, "")),
    Substances.pfpes \
        .with_starting_concentration(StartingConcentration(np.array([830]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 8.3, 2023, "")),
    Substances.pfhxs \
        .with_starting_concentration(StartingConcentration(np.array([54]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 8.3, 2023, "")),
    Substances.pfos \
        .with_starting_concentration(StartingConcentration(np.array([449]))) \
        .with_reference(Reference("Bundesministerium für Gesundheit 2023, Zeilmaker et al., 2018 ", 2.5, 2023, "")), 
    Substances.tfmsa \
        .with_starting_concentration(StartingConcentration(np.array([100]))) \
        .with_reference(Reference("No Reference - relative assessment", 1, 0, "")),
    Substances.benzo \
        .with_starting_concentration(StartingConcentration(np.array([44000]))) \
        .with_reference(Reference("Umweltbundesamt 2023 ", 3000, 2023, "")),
    Substances.cbz \
        .with_starting_concentration(StartingConcentration(np.array([5000]))) \
        .with_reference(Reference("UBA GOW (july 2020)", 300, 2020, "")),
]

AttributeError: 'Substance' object has no attribute 'with_starting_concentration'

## 2. Define the treatment train, 
The TreatmentTrain() definition uses a list of treatments as input. All implemented treatments can be called by "Treatments."

In [ ]:
base_tt = TreatmentTrain([
    Treatments.wwtt, # primary and secondary wastewater treatment
    Treatments.wwco, # additional wastewater treatment: coagulation 
    Treatments.dilsw, # dilution by surface water
    Treatments.npbk, # bank filtration
    Treatments.dilgw, # dilution by groundwater
    Treatments.dwae, # drinking water treatment: aeration
    Treatments.dwrf, # drinking water treatment: rapid filtration
    Treatments.dwac # drinking water treatemtn: activated carbon
])

## 3. Add mixture data characteristics (Optionally substance specific)
For each mixture process (prefix either "dil" or "sep"), site-specific characteristics need to be added. For mixtures which are not substance-dependent this can simply be done as shown above in the treatment train. If the concentration is specific for a substance, it can be added by looping through the substances and provide a list of mean background concentrations.

In [ ]:
# No background concentration in river and groundwater
x_m_river = 0.666
x_sd_river = x_m_river * 0.2
x_m_ground = 0.25
x_sd_ground = x_m_ground * 0.2
c_m_river = []
c_m_ground = []

river_mixture, groundwater_mixture = [], []
for i,s in enumerate(substances):
    river_mixture += [Mixture(x_m_river, x_sd_river, 0, 0)]
    groundwater_mixture  += [Mixture(x_m_ground, x_sd_ground, 0, 0)]

## 4. Add site-specific removal factors (optional)
In this example, on-site measurements for the removal by granular activated carbon as drinking water treatment  (dwac) were available. Per treatment step, site specific removal factors can be added. If this is done for a treatment, all substances defined above need to be listed in the same order, even if there is no data available. In this case an empty array can be provided to the RemovalPercent() definition.

In [ ]:
# between 10 000 and 20 000 bed volume
removals = {
     Treatments.dwac: {
        Substances.pfba: RemovalPercent(np.array([8, 7, 5, 8, 4, 4, 5])),
        Substances.pfpea: RemovalPercent(np.array([6, 4, 4, 15, 10, 15, 11])),
        Substances.pfhxa: RemovalPercent(np.array([7,4,9,26,25,27,32])),
        Substances.pfhpa: RemovalPercent(np.array([17,12,19,25,20,29,31])),
        Substances.pfoa: RemovalPercent(np.array([27,21,21,39,23,40,30])),
        Substances.pfna: RemovalPercent(np.array([30,17,23,35,18,59,39])),
        Substances.pfda: RemovalPercent(np.array([33,42,34,26,-8,74,34])),
        Substances.pfbs: RemovalPercent(np.array([-9,6,-12,33,26,32,29])),
        Substances.pfpes: RemovalPercent(np.array([31,21,30,42,36,41,38])),
        Substances.pfhxs: RemovalPercent(np.array([31,23,31,43,36,51,39])),
        Substances.pfos: RemovalPercent(np.array([33,27,28,42,18,69,34])),
        Substances.tfmsa: RemovalPercent(np.array([6,8,5,15,7,3,-3,4])),
        Substances.benzo: RemovalPercent(np.array([88,89,89,85,89,88,78,79])),
        Substances.cbz: RemovalPercent(np.array([99,98,99,98,97,94,99,99]))
     }
}

## 5. Create scenarios to be compared
In our example, we want to compare the exposure to different substances by the same treatment train. One scenario is defined as a unique combination of i) substance, ii) treatment train and iii) treatment properties (including mixture properties). Per substance 1 scenario will be created. By looping through the substances, all substance-specific information can be added.

In [ ]:
# create scenarios using treatment trains, substance properties and indput matrix
sc_baseline = []
for i,s in enumerate(substances):
        r = removals[Treatments.dwac][s]
        base_tt[-1].with_removal(r)
        base_tt[2].with_mixture(river_mixture[i])
        base_tt[4].with_mixture(groundwater_mixture[i])
        
        t = deepcopy(base_tt)
       
        sc_baseline += [Scenario(
            f"{s.id}",
            input_matrix = Matrices.rww,
            substance = s,
            treatment_train = t,
        )]



## 6. Run simulation

In [ ]:
result_baseline = []
for sc in sc_baseline:
    result_baseline += [sc.simulate_removal(n_runs = 10000)]

## 7. Plot results
### Profiles along the treatment train
The concentration and removal factors can be displayed with er_profiles() for selected scenarios (one single scenario is possible). Mixture processes can be excluded from the removal factor plot because the inflow of contaminated water sources can lead to high negative "removal factors" which cannot be compared to the remaining processes.
### Scenario comparison at the end of the treatment train
The spider plot compares the exceedance of the reference value in two or more scenarios. For this purpose, the upper half of the ditribution of the final concentration (50th to 99th quantile) is divided by the reference value. The resulting reference quotient is plotted in color scale.

In [ ]:
er_profiles(
    sim_results = [result_baseline[10], result_baseline[11],  result_baseline[13]],
    no_mixture = True,
    color_palette = ['#FF4A37', '#FC8604', '#2ED812'],
    reference_value = None
)
plt.show()

In [ ]:
spider_plot([result_baseline[10], result_baseline[11],  result_baseline[13]])
plt.show()

In [ ]:
spider_plot(result_baseline)
plt.show()